# **Laboratorio 2**

Derek Arreaga - 22537
Mónica Salvatierra - 22249

In [1]:
import numpy as np
import tensorflow as tf

Descargar dataset de MNIST y generar los conjuntos de entrenamiento y prueba

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


print("Entrenamiento:", x_train.shape, y_train.shape)
print("Prueba:", x_test.shape, y_test.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 10s 1us/step
Entrenamiento: (60000, 28, 28) (60000,)
Prueba: (10000, 28, 28) (10000,)


In [6]:
# Normalizar los valores de píxel a rango [0, 1]
x_train = x_train / 255.0
x_test = x_test / 255.0


In [8]:
print("Min pixel value:", x_train.min())
print("Max pixel value:", x_train.max())


Min pixel value: 0.0
Max pixel value: 1.0


## Preprocesamiento de datos

Crear un conjunto de validación a partir del conjunto de entrenamiento

In [7]:
from sklearn.model_selection import train_test_split

# Dividir x_train en 80% entrenamiento y 20% validación
x_train_new, x_val, y_train_new, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42)

print("x_train:", x_train_new.shape)
print("x_val:", x_val.shape)
print("x_test:", x_test.shape)


x_train: (48000, 28, 28)
x_val: (12000, 28, 28)
x_test: (10000, 28, 28)


1. Modificación del Ancho de la Red

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

history = model.fit(
    x_train_new, y_train_new,
    validation_data=(x_val, y_val),
    epochs=5
)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.8641 - loss: 0.4733 - val_accuracy: 0.9557 - val_loss: 0.1589
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9603 - loss: 0.1359 - val_accuracy: 0.9610 - val_loss: 0.1290
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9735 - loss: 0.0887 - val_accuracy: 0.9675 - val_loss: 0.1033
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9801 - loss: 0.0652 - val_accuracy: 0.9714 - val_loss: 0.0953
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9861 - loss: 0.0470 - val_accuracy: 0.9712 - val_loss: 0.0906


In [13]:
val_acc = history.history['val_accuracy'][-1]
print("Precisión de validación final:", round(val_acc * 100, 2), "%")


Precisión de validación final: 97.12 %


In [14]:
import time

start = time.time()

history = model.fit(
    x_train_new, y_train_new,
    validation_data=(x_val, y_val),
    epochs=5
)

end = time.time()
print("Tiempo total de entrenamiento:", round(end - start, 2), "segundos")


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9882 - loss: 0.0378 - val_accuracy: 0.9744 - val_loss: 0.0817
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.9904 - loss: 0.0306 - val_accuracy: 0.9753 - val_loss: 0.0853
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9938 - loss: 0.0222 - val_accuracy: 0.9759 - val_loss: 0.0867
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9949 - loss: 0.0179 - val_accuracy: 0.9764 - val_loss: 0.0845
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9959 - loss: 0.0150 - val_accuracy: 0.9767 - val_loss: 0.0900
Tiempo total de entrenamiento: 34.42 segundos


Modifique el tamaño de la capa escondida a 200 neuronas

In [16]:
model = keras.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(200, activation='relu'),  # ← cambio aquí
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

start = time.time()

history = model.fit(
    x_train_new, y_train_new,
    validation_data=(x_val, y_val),
    epochs=5
)

end = time.time()
tiempo = round(end - start, 2)

# Obtener precisión final en validación
val_acc = round(history.history['val_accuracy'][-1] * 100, 2)

print(f"\n Precisión de validación final: {val_acc}%")
print(f"Tiempo total de entrenamiento: {tiempo} segundos")

Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8760 - loss: 0.4358 - val_accuracy: 0.9586 - val_loss: 0.1408
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9635 - loss: 0.1220 - val_accuracy: 0.9688 - val_loss: 0.1036
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9779 - loss: 0.0778 - val_accuracy: 0.9693 - val_loss: 0.0992
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9826 - loss: 0.0567 - val_accuracy: 0.9767 - val_loss: 0.0838
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9889 - loss: 0.0361 - val_accuracy: 0.9728 - val_loss: 0.0973

 Precisión de validación final: 97.28%
Tiempo total de entrenamiento: 39.58 segundos


**¿Cómo cambia la precisión de validación del modelo?**

- Entrenando el modelo con una capa oculta de 200 neuronas y 5 épocas, el rendimiento no fue muy diferente al modelo baseque ocupaba 128 neuronas en su capa oculta. Para el modelo base, se obtuvo un accuracy del 97.12%, mientras que en el nuevo modelo, el accuracy fue de 97.28%.

**¿Cuánto tiempo tarda el algoritmo en entrenar?**

- El modelo base tardó 34.42 segundos en entrenarse, mientras el nuevo modelo con más neuronas tardó 39.58 segundos. Hubo una diferencia de 5.16 segundos. 

Experimente con diferentes tamaños de capa escondida (50, 100, 300, 500) y determine
cuál ofrece el mejor rendimiento.

In [17]:
# Distintos tamaños de capa oculta que quieres probar
neuronas_ocultas = [50, 100,300, 500]

# Para guardar resultados
resultados = []

for n in neuronas_ocultas:
    print(f"\nEntrenando modelo con {n} neuronas en la capa oculta...")

    # Definir modelo
    model = keras.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(n, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])

    # Compilar
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    # Medir tiempo
    start_time = time.time()

    # Entrenar
    history = model.fit(
        x_train_new, y_train_new,
        validation_data=(x_val, y_val),
        epochs=5,
        verbose=1
    )

    end_time = time.time()
    tiempo = round(end_time - start_time, 2)
    val_acc = round(history.history['val_accuracy'][-1] * 100, 2)

    resultados.append((n, val_acc, tiempo))
    print(f"→ Precisión de validación: {val_acc}% | Tiempo: {tiempo} segundos")


Entrenando modelo con 50 neuronas en la capa oculta...
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8424 - loss: 0.5710 - val_accuracy: 0.9420 - val_loss: 0.2039
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9474 - loss: 0.1827 - val_accuracy: 0.9570 - val_loss: 0.1500
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9611 - loss: 0.1343 - val_accuracy: 0.9605 - val_loss: 0.1336
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.9686 - loss: 0.1061 - val_accuracy: 0.9657 - val_loss: 0.1188
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9744 - loss: 0.0854 - val_accuracy: 0.9646 - val_loss: 0.1180
→ Precisión de validación: 96.46% | Tiempo: 12.79 segundos

Entrenando modelo con 100 neuronas en la capa oculta...
Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8592 - loss: 0.5037 - val_accuracy: 0.9445 - val_loss: 0.1842
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step -

La configuración que tuvo el mejor rendimiento fue el que contiene 300 neuronas en la capa oculta con un rendimiento de 97.82% y un tiempo de ejecución de 39.19 segundos.

2. Modificación de la Profundidad de la Red